In [1]:
import glob
import os.path
import warnings
from os.path import exists
import pandas as pd
from src.OfflineSlidingWindow import sliding_window
from src.ExtractFeatures import compute_features

# from PreprocessSignals import preprocessing_EDA, processing_labels, preprocessing_BVP, select_data

warnings.filterwarnings('ignore')

# set parameter
windowLength = 5 * 1000  # unit milliseconds
overlap = 0.5

----------
# Run Feature Extraction for Distracted Pedestrians data

In [2]:
path_sensor = "./Data/input/"
sensors = ['Accelerometer']
inputs = []

In [17]:
error_protocol = []

# run through all participants here
for participant_filepath in glob.iglob(path_sensor + "AR/", recursive=True):
    try:
        components = os.path.normpath(participant_filepath)
        components = components.split(os.path.sep)
        participant = components[-1]
        print(participant)

        # read in labels
        ground_truth = pd.read_csv('{}Annotation.csv'.format(participant_filepath))
        ground_truth['time'] = ground_truth['time'].astype('float64')
        ground_truth.set_index('time', inplace=True)

        # get information when HRI began and stopped for sliding window
        start_ts = ground_truth.first_valid_index()
        end_ts = ground_truth.last_valid_index()

        # variable to store extracted features
        extractedFeatures = pd.DataFrame()
        # run through single sensors
        flag = True
        for sensor in sensors:
            # check if filesize empty
            if os.stat(participant_filepath + sensor + ".csv").st_size == 0: continue

            # read in sensor data
            sensor_data = pd.read_csv(participant_filepath + sensor + ".csv")
            sensor_data['time'] = sensor_data['time'].astype('float64')
            sensor_data = sensor_data.drop(['seconds_elapsed'], axis=1)
            display(sensor_data)
            # use only E4 data needed
            sensor_data = sensor_data[(sensor_data.time >= start_ts) & (sensor_data.time <= end_ts)]

            #check if dataframe is empty
            if sensor_data.empty:
                continue
            sensor_data.set_index('time', inplace=True)  # drop=False?
            # within this notfication do windowing and calculate features
            column = []
            if sensor == "Accelerometer":
                column = ['x', 'y', 'z']
            elif sensor == "Gyroscope":
                column = ['x', 'y', 'z']
            else:
                column = [str(sensor)]
            # start feature extraction
            display(column)
            result = sliding_window(sensor_data, column, windowLength, overlap, start_ts)
            # create one big dataframe containing all sensor data
            # if flag:
            #     extractedFeatures = result
            #     flag = False
            # else:
            #     extractedFeatures = pd.merge_ordered(extractedFeatures, result, on=['window_ts'], how='outer')
            extractedFeatures.to_csv(participant_filepath + participant + "_features.csv", sep=',', index=False)
            print(participant_filepath)

    except:
        error_protocol.append(participant_filepath)
        continue

AR


,time,lux
0,1.670789e+18,0.0000
1,1.670789e+18,1.4125
2,1.670789e+18,2.9875
3,1.670789e+18,4.4125
4,1.670789e+18,9.1125
5,1.670789e+18,0.0000
6,1.670789e+18,2.5250
7,1.670789e+18,0.0000
8,1.670789e+18,2.8000
9,1.670789e+18,0.0000


['Light']

In [18]:
error_protocol = []

# run through all participants here
for participant_filepath in glob.iglob(path_sensor + "AR/", recursive=True):
    try:
        components = os.path.normpath(participant_filepath)
        components = components.split(os.path.sep)
        participant = components[-1]
        print(participant)

        # read in labels
        ground_truth = pd.read_csv('{}Annotation.csv'.format(participant_filepath))
        ground_truth['time'] = ground_truth['time'].astype('float64')
        ground_truth.set_index('time', inplace=True)

        # get information when HRI began and stopped for sliding window
        start_ts = ground_truth.first_valid_index()
        end_ts = ground_truth.last_valid_index()

        # variable to store extracted features
        extractedFeatures = pd.DataFrame()
        # run through single sensors
        flag = True
        for sensor in sensors:
            # check if filesize empty
            if os.stat(participant_filepath + sensor + ".csv").st_size == 0: continue

            # read in sensor data
            sensor_data = pd.read_csv(participant_filepath + sensor + ".csv")
            sensor_data['time'] = sensor_data['time'].astype('float64')
            sensor_data = sensor_data.drop(['seconds_elapsed'], axis=1)
            display(sensor_data)
            # use only E4 data needed
            sensor_data = sensor_data[(sensor_data.time >= start_ts) & (sensor_data.time <= end_ts)]

            #check if dataframe is empty
            if sensor_data.empty:
                continue
            sensor_data.set_index('time', inplace=True)  # drop=False?
            # within this notfication do windowing and calculate features
            column = []
            if sensor == "Accelerometer":
                column = ['x', 'y', 'z']
            elif sensor == "Gyroscope":
                column = ['x', 'y', 'z']
            else:
                column = [str(sensor)]
            # start feature extraction
            display(column)

            # if sensor_data.shape[0] < 1:
            #     continue
            # if type(overlap) != type(0.0) or type(windowLength) != type(0):
            #     raise Exception('**Error** type(overlap) must be float and type(windowsize) must be int')
            # if overlap > windowLength:
            #     raise Exception('**Error** overlap is larger than windowsize...')
            #
            # if "lux" in column:
            #     # Creating chunks
            #     suffix = ['slope', 'sqrtSlope', 'sqrtIntersect', 'thirdSqrtIntersect']
            #     header1 = ['{}_{}'.format(col, suf) for col in column for suf in suffix]
            #     suffix = ['var', 'std', 'skew', 'rms', 'min', 'max', 'mean', 'median']
            #     header2 = ['{}_{}'.format(col, suf) for col in column for suf in suffix]
            #     header = header1 + header2 + ['window_ts']
            #
            # if  label:
            #     header =  header + ["label"]
            #
            # result = pd.DataFrame(columns=header)
            #
            # # set timing parameters
            # # step is the time the window slides forward
            # step = int(windowLength * (1 - overlap))
            # if start_ts != None:
            #     start = start_ts
            # else:
            #     start = sensor_data['time'].iloc[0]
            # # grap last instance of dataset as end time
            # end = sensor_data.last_valid_index()
            #
            # # get windows
            # while start < end:
            #     row_result = []
            #     window = sensor_data[(sensor_data.index >= start) & (start + windowLength >= sensor_data.index)]
            #
            #     # does window contain data
            #     if window.shape[0] > 1:  # needs more than one element in window
            #         if 'lux' in column:
            #             # row_result = get_IBI_features(window)
            #             row_result['window_ts'] = start
            #             # labels - labels hinzufügen
            #             if label:
            #                 row_result.append(pd.value_counts(window['label']).idxmax())
            #             result = result.append(row_result)
            #         else:
            #             # start statistical feature calculation
            #             row_result = compute_features(window, column)
            #             row_result.append(start)
            #             # labels - labels hinzufügen
            #             if label:
            #                 row_result.append(pd.value_counts(window['label']).idxmax())
            #             # add window to final result
            #             result.loc[len(result)] = row_result
            #     start = start + step
            # return result
            result = sliding_window(sensor_data, column, windowLength, overlap, start_ts)
            # create one big dataframe containing all sensor data
            if flag:
                extractedFeatures = result
                flag = False
            else:
                extractedFeatures = pd.merge_ordered(extractedFeatures, result, on=['window_ts'], how='outer')
            extractedFeatures.to_csv(participant_filepath + participant + "_features.csv", sep=',', index=False)
            print(participant_filepath)

    except:
        error_protocol.append(participant_filepath)
        continue

AR


,time,lux
0,1.670789e+18,0.0000
1,1.670789e+18,1.4125
2,1.670789e+18,2.9875
3,1.670789e+18,4.4125
4,1.670789e+18,9.1125
5,1.670789e+18,0.0000
6,1.670789e+18,2.5250
7,1.670789e+18,0.0000
8,1.670789e+18,2.8000
9,1.670789e+18,0.0000


['Light']

In [2]:
path_sensor = "./Data/input/"
sensors = ['Accelerometer']
inputs = []
# set parameter
windowLength = 5 * 10  # unit milliseconds
overlap = 0.5

In [3]:
error_protocol = []

# run through all participants here
for participant_filepath in glob.iglob(path_sensor + "AR/", recursive=True):
    try:
        components = os.path.normpath(participant_filepath)
        components = components.split(os.path.sep)
        participant = components[-1]
        print(participant)

        # read in labels
        ground_truth = pd.read_csv('{}Annotation.csv'.format(participant_filepath))
        ground_truth['time'] = ground_truth['time'].astype('float64')
        ground_truth.set_index('time', inplace=True)

        # get information when HRI began and stopped for sliding window
        start_ts = ground_truth.first_valid_index()
        end_ts = ground_truth.last_valid_index()

        # variable to store extracted features
        extractedFeatures = pd.DataFrame()
        # run through single sensors
        flag = True
        for sensor in sensors:
            # check if filesize empty
            if os.stat(participant_filepath + sensor + ".csv").st_size == 0: continue

            # read in sensor data
            sensor_data = pd.read_csv(participant_filepath + sensor + ".csv")
            sensor_data['time'] = sensor_data['time'].astype('float64')
            sensor_data = sensor_data.drop(['seconds_elapsed'], axis=1)
            # display(sensor_data)
            # use only E4 data needed
            sensor_data = sensor_data[(sensor_data.time >= start_ts) & (sensor_data.time <= end_ts)]

            #check if dataframe is empty
            if sensor_data.empty:
                continue
            sensor_data.set_index('time', inplace=True)  # drop=False?
            # within this notfication do windowing and calculate features
            column = []
            if sensor == "Accelerometer":
                column = ['x', 'y', 'z']
            elif sensor == "Gyroscope":
                column = ['x', 'y', 'z']
            else:
                column = [str(sensor)]
            # start feature extraction
            # display(column)
            segments = sensor_data.rolling(window=windowLength, center=True).agg(['min', 'max', 'sum', 'mean', 'std'])
            segments.columns = ['-'.join(tup).rstrip('-') for tup in segments.columns.values]
            segments = pd.concat([sensor_data.iloc[:, -1], segments], axis=1)
            # segments = segments.dropna()
            # segments = segments[::(int)(window_size * overlap)]
            percentage = (int)((windowLength)-(windowLength * overlap))
            segments = segments[::percentage]
            extractedFeatures = segments
            display(extractedFeatures)
            # result = sliding_window(sensor_data, column, windowLength, overlap, start_ts)
            # create one big dataframe containing all sensor data
            # if flag:
            #     extractedFeatures = result
            #     flag = False
            # else:
            #     extractedFeatures = pd.merge_ordered(features_df, result, on=['time'], how='outer')
            extractedFeatures.to_csv(participant_filepath + participant + "_features.csv", sep=',', index=False)
            # print(participant_filepath)

    except:
        error_protocol.append(participant_filepath)
        continue

In [9]:
# get feature names from the file features.txt
features = list()
with open('Data/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

print('No of Features: {}'.format(len(features)))

No of Features: 561


In [10]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

In [ ]:
# https://github.com/arijitiiest/UCI-Human-Activity-Recognition/blob/master/Data-preprocessing.ipynb